In [20]:
import kfp
from kfp.components import load_component_from_file
import kfp.dsl as dsl
from kfp.v2.dsl import Model, Dataset, Artifact

init_op = load_component_from_file("components/init_artifacts/component_SDKv2.yaml")
train_op = load_component_from_file("components/train/component_SDKv2.yaml")

@dsl.pipeline(
    name="KWS-train-test-pipe"
)
def pipeline(
    config_file_url: str,
    dataset_path: str,
    model_S3_bucket: str,
):
    #initialize artifacts

    #actual pipeline steps
    train_task = train_op(
        config,
        dataset, 
        model)

kfp.compiler.Compiler(mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE).compile(
    pipeline_func=pipeline,
    package_path='pipeline.yaml')


Unexpected exception formatting exception. Falling back to standard exception


/home/lebugcat/miniconda3/envs/kws-dev-env/lib/python3.9/site-packages/kfp/compiler/compiler.py:79: UserWarning: V2_COMPATIBLE execution mode is at Beta quality. Some pipeline features may not work as expected.
  pipeline_name: Name of the pipeline.
Traceback (most recent call last):
  File "/home/lebugcat/miniconda3/envs/kws-dev-env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_50565/3233065213.py", line 30, in <cell line: 30>
    kfp.compiler.Compiler(mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE).compile(
  File "/home/lebugcat/miniconda3/envs/kws-dev-env/lib/python3.9/site-packages/kfp/compiler/compiler.py", line 1175, in compile
    self._create_and_write_workflow(
  File "/home/lebugcat/miniconda3/envs/kws-dev-env/lib/python3.9/site-packages/kfp/compiler/compiler.py", line 1227, in _create_and_write_workflow
    workflow = self._create_workflow(pipeline_func, pi

In [19]:
!pip install kfp==1.8.13

  Using cached kfp-1.8.13-py3-none-any.whl
  Using cached google_cloud_storage-1.44.0-py2.py3-none-any.whl (106 kB)
  Using cached kfp_server_api-1.8.4-py3-none-any.whl
  Attempting uninstall: kfp-server-api
    Found existing installation: kfp-server-api 2.0.0a3
    Uninstalling kfp-server-api-2.0.0a3:
      Successfully uninstalled kfp-server-api-2.0.0a3
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.5.0
    Uninstalling google-cloud-storage-2.5.0:
      Successfully uninstalled google-cloud-storage-2.5.0
  Attempting uninstall: kfp
    Found existing installation: kfp 2.0.0b3
    Uninstalling kfp-2.0.0b3:
      Successfully uninstalled kfp-2.0.0b3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kserve 0.8.0 requires google-api-core==1.29.0, but you have google-api-core 2.8.2 which is incompatible.
kserve 

In [12]:
from kfp.v2.dsl import Input, Output, OutputPath, Dataset, Model
from kfp.v2.dsl import component
@component(
    output_component_file="component2.yaml"
)
def train(
    # Use Input[T] to get a metadata-rich handle to the input artifact of type `Dataset`.
    dataset: Input[Dataset],
    # Use Output[T] to get a metadata-rich handle to the output artifact of type `Dataset`.
    model: Output[Model],
    # An input parameter of type int.
    num_steps: int,
    # An output parameter of type str.
    output_message_path: OutputPath(str),
):
    """Dummy Training step."""

    with open(dataset.path, 'r') as input_file:
        dataset_contents = input_file.read()

    with open(model.path, 'w') as output_file:
        for i in range(num_steps):
            output_file.write(f'Step {i}\n=====\n')

    # Model artifact has a `.metadata` dictionary
    # to store arbitrary metadata for the output artifact.
    model.metadata['accuracy'] = 0.9

    with open(output_message_path, 'w') as output_file:
        output_file.write('Model trained successfully.')

# import kfp
# kfp.components.create_component_from_func(
#     func= train,
#     output_component_file="test.yaml"
# )

In [8]:
!pip install kfp==2.0.0b3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.3/334.3 kB 886.6 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for kfp: filename=kfp-2.0.0b3-py3-none-any.whl size=476367 sha256=4fbeda29bcf00536e4ba3b37ba39ff1b03b1dd742adc3e6fc8834f5d002b9d14
  Stored in directory: /home/lebugcat/.cache/pip/wheels/65/fc/20/b9dc05d38381c134023598e898871f5de2047ad438e7f7f93f
Successfully built kfp
  Attempting uninstall: kfp
    Found existing installation: kfp 2.0.0b2
    Uninstalling kfp-2.0.0b2:
      Successfully uninstalled kfp-2.0.0b2
